In [1]:
# Pandas dataframe
import pandas as pd
from pandas import DataFrame

# Scikit-Learn
import sklearn 
from sklearn import datasets, linear_model, metrics, tree
from sklearn.model_selection import train_test_split, LeaveOneOut, KFold, cross_validate, RandomizedSearchCV, GridSearchCV
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error, accuracy_score

# Models
from sklearn.linear_model import LinearRegression, ElasticNet, Lasso, Ridge
from sklearn.ensemble import (BaggingRegressor, ExtraTreesRegressor, GradientBoostingRegressor, 
RandomForestRegressor, AdaBoostRegressor)
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import ElasticNet, Lasso,  RidgeCV
from sklearn.neighbors import KNeighborsRegressor
        
import joblib
from matplotlib import pyplot as plt
import numpy as np
import os
from tqdm import tqdm

In [2]:
df_g = pd.read_csv("dataset_scaled_g.csv")
df_g.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 127 entries, 0 to 126
Data columns (total 29 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0.2           127 non-null    int64  
 1   Unnamed: 0.1           127 non-null    int64  
 2   Unnamed: 0             127 non-null    int64  
 3   atomic_number          127 non-null    float64
 4   radius_pm              127 non-null    float64
 5   oxid_state             127 non-null    float64
 6   relativistic           127 non-null    float64
 7   metal_amount_mmol      127 non-null    float64
 8   num of group           127 non-null    float64
 9   spcaer                 127 non-null    float64
 10  anchoring_group        127 non-null    float64
 11  chirality type         127 non-null    float64
 12  hydrogen bond          127 non-null    float64
 13  aromaticity            127 non-null    float64
 14  ligand_amount_mmol     127 non-null    float64
 15  redu_n

In [3]:
input_col = [col for col in df_g.columns]
input_col.remove('Unnamed: 0')
input_col.remove('Unnamed: 0.1')
input_col.remove('Unnamed: 0.2')
input_col.remove('g_factor*10^4')
input_col

['atomic_number',
 'radius_pm',
 'oxid_state',
 'relativistic',
 'metal_amount_mmol',
 'num of group',
 'spcaer',
 'anchoring_group',
 'chirality type',
 'hydrogen bond',
 'aromaticity',
 'ligand_amount_mmol',
 'redu_num',
 'reductant_amount_mmol',
 'protic1',
 'polarity index1',
 'aromaticity1',
 'sol1_vol_mL',
 'protic2',
 'polarity index 2',
 'aromaticity2',
 'solv2_vol_mL',
 'time_min',
 'temp_c',
 'size_nm']

In [4]:
output_col = ['g_factor*10^4']

X = df_g[input_col]

Y = df_g[output_col]

In [5]:
# Splitting dataset for training
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.15, random_state=45, shuffle=True)

## Bagging

In [6]:
min_mae = 99999

min_i, min_j, min_k  = 0, 0, 0

for i in tqdm(range(1, 20)):
    for j in range(1, 20):
        for k in range(2, 36, 2):
            
            B_regr = BaggingRegressor(base_estimator=DecisionTreeRegressor(max_depth=i),
                                      n_estimators=j,
                                      random_state=k)
            
            B_regr.fit(X_train, np.ravel(Y_train))
            
            B_Y_pred = B_regr.predict(X_test)
            
            mae = mean_absolute_error(Y_test, B_Y_pred)
            
            if (min_mae > mae):

                min_mae = mae
                min_i = i
                min_j = j
                min_k = k
            
print(min_mae, min_i, min_j, min_k)

100%|██████████| 19/19 [02:00<00:00,  6.33s/it]

2.1366805286070853 7 3 18


## Random Forest

In [7]:
min_mae = 99999
min_i, min_j, min_k = 0, 0, 0
for i in tqdm(range(1, 26)):
    for j in range(1, 26):
        for k in range(2, 23, 2):
            RF_regr = RandomForestRegressor(max_depth=i, 
                                            n_estimators=j, 
                                            max_features=k,
                                            random_state=45
                                                )
            RF_regr.fit(X_train, np.ravel(Y_train))
            RF_Y_pred = RF_regr.predict(X_test)

            mae = mean_absolute_error(Y_test, RF_Y_pred)
            if (min_mae > mae):
                min_mae = mae
                min_i = i
                min_j = j
                min_k = k
            
print(min_mae, min_i, min_j, min_k)

100%|██████████| 25/25 [01:58<00:00,  4.72s/it]

1.6752124282416667 8 1 16


## Decision Tree

In [8]:
min_mae = 99999

min_i, min_j, min_k  = 0, 0, 0

for i in tqdm(range(1, 26)):
    for j in range(1, 23):
        for k in range(2, 45, 1):
            
            DT_regr = DecisionTreeRegressor(max_depth=i,
                                max_features=j,
                                random_state=k)
            
            DT_regr.fit(X_train, Y_train)

            DT_Y_pred = DT_regr.predict(X_test)

            mae = mean_absolute_error(Y_test, DT_Y_pred)
            
            if (min_mae > mae):
                min_mae = mae
                min_i = i
                min_j = j
                min_k = k
            
print(min_mae, min_i, min_j, min_k)

100%|██████████| 25/25 [01:18<00:00,  3.15s/it]

1.5933473270011906 14 1 8


## Extra Trees

In [9]:
min_mae = 99999
min_i, min_j, min_k = 0, 0, 0
for i in tqdm(range(1, 30)):
    for j in range(1, 23):
        for k in range(2, 40, 1):
            ET_regr = ExtraTreesRegressor(n_estimators=i, 
                                            max_features=j,
                                            random_state=k
                                                )
            ET_regr.fit(X_train, np.ravel(Y_train))
            ET_Y_pred = ET_regr.predict(X_test)

            mae = mean_absolute_error(Y_test, ET_Y_pred)
            if (min_mae > mae):
                min_mae = mae
                min_i = i
                min_j = j
                min_k = k
            
print(min_mae, min_i, min_j, min_k)

100%|██████████| 29/29 [05:13<00:00, 10.82s/it]

1.5263687329000002 2 16 14


## AdaBoost

In [10]:
min_mae = 99999
min_i, min_j = 0, 0
for i in tqdm(range(1, 30)):
    for j in range(1, 30):
        AB_regr = AdaBoostRegressor(n_estimators=i, loss='exponential', learning_rate=j)
        AB_regr.fit(X_train, np.ravel(Y_train))
        AB_Y_pred = AB_regr.predict(X_test)
        AB_mae = mean_absolute_error(Y_test, AB_Y_pred)
        if (min_mae > mae):
                min_mae = mae
                min_i = i
                min_j = j
print(min_mae, min_i, min_j)

100%|██████████| 29/29 [00:14<00:00,  2.06it/s]

2.797632417637931 1 1


## Gradient Boosting

In [11]:
'''
min_mae = 999
min_i, min_j, min_k, min_l = 0, 0, 0.0, 0
for i in tqdm(range(300, 450, 10)):
    for j in range(2, 30, 2):
        for k in np.arange(0.06, 0.22, 0.02):
            for l in range(2, 22, 2):
                GB_regr = GradientBoostingRegressor(n_estimators=i, max_depth=j, learning_rate=k, random_state=l)
                GB_regr.fit(X_train, np.ravel(Y_train))
                GB_Y_pred = GB_regr.predict(X_test)

                mae = mean_absolute_error(Y_test, GB_Y_pred)
                if (min_mae > mae):
                    min_mae = mae
                    min_i = i
                    min_j = j
                    min_k = k
                    min_l = l

print(min_mae, min_i, min_j, min_k, min_l)
'''

'\nmin_mae = 999\nmin_i, min_j, min_k, min_l = 0, 0, 0.0, 0\nfor i in tqdm(range(300, 450, 10)):\n    for j in range(2, 30, 2):\n        for k in np.arange(0.06, 0.22, 0.02):\n            for l in range(2, 22, 2):\n                GB_regr = GradientBoostingRegressor(n_estimators=i, max_depth=j, learning_rate=k, random_state=l)\n                GB_regr.fit(X_train, np.ravel(Y_train))\n                GB_Y_pred = GB_regr.predict(X_test)\n\n                mae = mean_absolute_error(Y_test, GB_Y_pred)\n                if (min_mae > mae):\n                    min_mae = mae\n                    min_i = i\n                    min_j = j\n                    min_k = k\n                    min_l = l\n\nprint(min_mae, min_i, min_j, min_k, min_l)\n'

## Others

In [12]:
REGRESSIONS = {
    "K-nn": KNeighborsRegressor(),                          
    "Ridge": RidgeCV(),
    "Lasso": Lasso(),
    "ElasticNet": ElasticNet(random_state=0),
                }
# mean absolute error is used to evaluate the performance of all regressions.


for name, reg in REGRESSIONS.items():     
    reg.fit(X_train, Y_train)                 
    Y_pred = pd.DataFrame(reg.predict(X_test))
    
    print(name)
    
    mae = mean_absolute_error(Y_test, Y_pred)
    
    print(' MAE for Abs is ', mae, '\n')

K-nn
 MAE for Abs is  3.54779198761 

Ridge
 MAE for Abs is  2.970336959193324 

Lasso
 MAE for Abs is  3.556298353799378 

ElasticNet
 MAE for Abs is  3.4389756472777444 



## Saving the best model

In [14]:
#Best Model
ET_regr = ExtraTreesRegressor(n_estimators=2,
                                            max_features=16,
                                            random_state=14
                                                )
ET_regr.fit(X_train, np.ravel(Y_train))
ET_Y_pred = ET_regr.predict(X_test)

joblib.dump(ET_regr, "./model_aug_g_ExtraTrees.joblib")

['./model_aug_g_ExtraTrees.joblib']